In [1]:
#params

region='GS'


In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(280)
cluster

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.2:36269 Dashboard: http://172.30.100.2:33209/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8606550     hsw24 make_pro albert7a PD       0:00      1 (Resources)
           8606551     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8606552     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8606553     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8606554     hsw24 make_pro albert7a PD       0:00      1 (Resources)
           8606555     hsw24 make_pro albert7a PD       0:00      1 (Resources)
           8606546     hsw24 make_pro albert7a  R       0:07      1 n2501
           8606547     hsw24 make_pro albert7a  R       0:07      1 n2173
           8606548     hsw24 make_pro albert7a  R       0:07      1 n2178
           8606549     hsw24 make_pro albert7a  R       0:07      1 n2187


In [5]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

112


In [6]:
## path for mdules

import sys
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/xscale")
import xscale

sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData

import os
import sys
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean
import glob
import time

%matplotlib inline


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [7]:
if region == 'EU':
    imin=970
    imax=1213
    jmin=651
    jmax=1224
if region == 'GS':
    imin=195
    imax=699
    jmin=572
    jmax=889

In [8]:
config='NACHOS12.L75'
freq='1d'

#!ls $datadir
gridfile='/scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_coordinates.nc'
maskfile='/scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_MAA40_mesh_mask.nc'

In [11]:
dsgrid=xr.open_dataset(gridfile)
dsmask=xr.open_dataset(maskfile)
navlat= dsgrid['gphit']
navlon= dsgrid['glamt']


In [12]:
def compute_TKE(u2mean,v2mean):
    ua=0.5*(u2mean+u2mean.shift(x=1))
    va=0.5*(v2mean+v2mean.shift(y=1))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    tke=0.5*(uat+vat)
    return tke


In [13]:
def compute_EKE(u2,v2,u2mean,v2mean):
    ua=0.5*((-1*u2+u2mean)+(-1*u2.shift(x=1)+u2mean.shift(x=1)))
    va=0.5*((-1*v2+v2mean)+(-1*v2.shift(y=1)+v2mean.shift(y=1)))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    eke=0.5*(uat+vat)
    return eke


In [14]:
month_name={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
def plot_ratio_eke_tke_monthly(var,loncrs,latcrs,lon,lat,hpvarm,m,mm,config,case):
    if region == 'EU':
        fig, ax = plt.subplots(1,1,figsize=(10,15))
    if region == 'GS':
        fig, ax = plt.subplots(1,1,figsize=(15,10))
    ax = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=0))
    ax.autoscale(tight=True)
    cont=np.isnan(hpvarm)
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    
    ax.tick_params('both',labelsize=25)

    pcolor = ax.pcolormesh(loncrs,latcrs,ma.masked_invalid(var),cmap='rainbow',vmin=0,vmax=1,alpha=1)
    ax.contour(lon,lat,cont,alpha=0.5,linewidth=0.000001,antialiased=True,colors='black')
    cbar = plt.colorbar(pcolor,orientation='horizontal',pad=0.1)
    cbar.ax.tick_params(labelsize=35)
    cbar.ax.tick_params(labelsize=25)
    cbar.set_label('Ratio of eke over tke in '+str(month_name[m])+' for '+config+'-'+case,fontsize=15)
    plt.savefig('Plots/'+config+'-'+case+'_y'+year+'m'+mm+'_ratio_eke_tke.png')
  

In [15]:
def plot_ratio(month,year):
    if year == '2010':
        case='MAA4001'
    else:
        case='MAA4002'
    mm = "{:02d}".format(month)
    datadir='/scratch/cnt0024/hmg2840/albert7a/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
    filesU=sorted(glob.glob(datadir+'/'+year+'/'+str(config)+'-'+str(case)+'_y'+str(year)+'m'+str(mm)+'d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'/'+year+'/'+str(config)+'-'+str(case)+'_y'+str(year)+'m'+str(mm)+'d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':10,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':10,'depthv':1})
    u=dsU.vozocrtx[:,:,jmin:jmax,imin:imax]
    v=dsV.vomecrty[:,:,jmin:jmax,imin:imax]
    mask=dsmask.tmask[0,:,jmin:jmax,imin:imax]
    umean=u.mean(dim='time_counter')
    vmean=v.mean(dim='time_counter')
    umean2=umean*umean
    vmean2=vmean*vmean
    u2=u*u
    v2=v*v
    u2mean=u2.mean(dim='time_counter')
    v2mean=v2.mean(dim='time_counter')
    tke=compute_TKE(u2,v2)
    eke=compute_EKE(umean2,vmean2,u2mean,v2mean)
    tkemean=tke.mean(dim='time_counter')
    ratio=eke/tkemean
    hpcurl=ratio
    hpcurl2 = hpcurl ** 2
    hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=12)
    hpcurl2mc = crs.return_ravel(np.asarray(hpcurl2m))
    hpcurl2mcm = np.mean(hpcurl2mc,axis=-3)
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)
    plot_ratio_eke_tke_monthly(ratio,loncrs,latcrs,lon,lat,hpvarm,mm,month,config,case)
        

In [16]:
year='2012'
#for month in [3,9]:
for month in [9]:
    plot_ratio(month,year)

shape and coarsening factors are not compatible


IndexError: tuple index out of range

In [ ]:
#test if workers still active
print(len(client.scheduler_info()["workers"]))